In [ ]:
!pip install -r '/content/drive/MyDrive/session_2/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 9.7 MB/s eta 0:00:00
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.2/39.2 MB 27.4 MB/s eta 0:

In [ ]:
# Import required libraries for advanced RAG
import os
from pathlib import Path
from typing import Dict, List, Optional, Any
from pydantic import BaseModel, Field

# Core LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever

# Vector store
from llama_index.vector_stores.lancedb import LanceDBVectorStore

# Embeddings and LLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

# Advanced RAG components (we'll use these in the assignments)
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.response_synthesizers import TreeSummarize, Refine, CompactAndRefine
from llama_index.core.output_parsers import PydanticOutputParser

print("✅ Advanced RAG libraries imported successfully!")

✅ Advanced RAG libraries imported successfully!


In [ ]:
import os
from pathlib import Path
from typing import Dict, List, Optional, Any
from pydantic import BaseModel, Field

# Core LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever

# Vector store
from llama_index.vector_stores.lancedb import LanceDBVectorStore

# Embeddings and LLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

# Advanced RAG components (we'll use these in the assignments)
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.response_synthesizers import TreeSummarize, Refine, CompactAndRefine
from llama_index.core.output_parsers import PydanticOutputParser

# For accessing Colab secrets
from google.colab import userdata

print("✅ Advanced RAG libraries imported successfully!")

# Configure Advanced RAG Settings (Using OpenRouter)
def setup_advanced_rag_settings():
    """
    Configure LlamaIndex with optimized settings for advanced RAG.
    Uses local embeddings and OpenRouter for LLM operations.
    """
    # Check for OpenRouter API key
    api_key = userdata.get("OPENROUTER_API_KEY") # Use userdata.get() for Colab secrets
    if not api_key:
        print("⚠️  OPENROUTER_API_KEY not found - LLM operations will be limited")
        print("   You can still complete postprocessor and retrieval exercises")
    else:
        print("✅ OPENROUTER_API_KEY found - full advanced RAG functionality available")

        # Configure OpenRouter LLM
        Settings.llm = OpenRouter(
            api_key=api_key,
            model="gpt-4o",
            temperature=0.1  # Lower temperature for more consistent responses
        )

    # Configure local embeddings (no API key required)
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="BAAI/bge-small-en-v1.5",
        trust_remote_code=True
    )

    # Advanced RAG configuration
    Settings.chunk_size = 512  # Smaller chunks for better precision
    Settings.chunk_overlap = 50

    print("✅ Advanced RAG settings configured")
    print("   - Chunk size: 512 (optimized for precision)")
    print("   - Using local embeddings for cost efficiency")
    print("   - OpenRouter LLM ready for response synthesis")

# Setup the configuration
setup_advanced_rag_settings()

✅ Advanced RAG libraries imported successfully!
✅ OPENROUTER_API_KEY found - full advanced RAG functionality available
✅ Advanced RAG settings configured
   - Chunk size: 512 (optimized for precision)
   - Using local embeddings for cost efficiency
   - OpenRouter LLM ready for response synthesis


In [ ]:
# Setup: Create index from Assignment 1 (reuse the basic functionality)
def setup_basic_index(data_folder: str = "/content/drive/MyDrive/session_2/data", force_rebuild: bool = False):
    """
    Create a basic vector index that we'll enhance with advanced techniques.
    This reuses the concepts from Assignment 1.
    """
    # Create vector store
    vector_store = LanceDBVectorStore(
        uri="./advanced_rag_vectordb",
        table_name="documents"
    )

    # Load documents
    if not Path(data_folder).exists():
        print(f"❌ Data folder not found: {data_folder}")
        return None

    reader = SimpleDirectoryReader(input_dir=data_folder, recursive=True)
    documents = reader.load_data()

    # Create storage context and index
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        show_progress=True
    )

    print(f"✅ Basic index created with {len(documents)} documents")
    print("   Ready for advanced RAG techniques!")
    return index

# Create the basic index
print("📁 Setting up basic index for advanced RAG...")
index = setup_basic_index()

if index:
    print("🚀 Ready to implement advanced RAG techniques!")
else:
    print("❌ Failed to create index - check data folder path")

📁 Setting up basic index for advanced RAG...


100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 39.5MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Parsing nodes:   0%|          | 0/42 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/90 [00:00<?, ?it/s]

✅ Basic index created with 42 documents
   Ready for advanced RAG techniques!
🚀 Ready to implement advanced RAG techniques!


In [ ]:
from llama_index.core.postprocessor import SimilarityPostprocessor

def create_query_engine_with_similarity_filter(index, similarity_cutoff: float = 0.3, top_k: int = 10):
    """
    Create a query engine that filters results based on similarity scores.

    Args:
        index: Vector index to query
        similarity_cutoff: Minimum similarity score (0.0 to 1.0)
        top_k: Number of initial results to retrieve before filtering

    Returns:
        Query engine with similarity filtering
    """
    # Create similarity postprocessor with the cutoff threshold
    similarity_processor = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)

    # Create query engine with similarity filtering
    query_engine = index.as_query_engine(
        similarity_top_k=top_k,
        node_postprocessors=[similarity_processor]
    )

    return query_engine

# Testing the function (moved outside the function definition)
if index:
    filtered_engine = create_query_engine_with_similarity_filter(index, similarity_cutoff=0.3)

    if filtered_engine:
        print("✅ Query engine with similarity filtering created")

        # Test query
        test_query = "What are the benefits of AI agents?"
        print(f"\n🔍 Testing query: '{test_query}'")

        response = filtered_engine.query(test_query)
        print(f"📝 Response: {response}")
    else:
        print("❌ Failed to create filtered query engine")
else:
    print("❌ No index available - run previous cells first")

✅ Query engine with similarity filtering created

🔍 Testing query: 'What are the benefits of AI agents?'
📝 Response: AI agents offer several benefits, including the enhancement of language model capabilities to solve real-world problems with robust problem-solving skills. They can operate in complex environments, make autonomous decisions, and assist humans in various tasks. AI agents are proficient in reasoning and planning, allowing them to swiftly learn new tasks and make informed decisions even in uncertain conditions. They can also employ multiple tools to address complex issues, interact with external data sources, and access information from APIs. Additionally, multi-agent architectures can improve performance by enabling parallel task execution and collaboration among agents, especially when multiple distinct execution paths are required.


In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize

def create_query_engine_with_tree_summarize(index, top_k: int = 5):
    """
    Create a query engine that uses TreeSummarize for comprehensive responses.

    Args:
        index: Vector index to query
        top_k: Number of results to retrieve

    Returns:
        Query engine with TreeSummarize synthesis
    """
    # Create TreeSummarize response synthesizer
    tree_synthesizer = TreeSummarize()

    # Create query engine with the synthesizer
    query_engine = index.as_query_engine(
        similarity_top_k=top_k,
        response_synthesizer=tree_synthesizer,
    )

    return query_engine
# Test the function
if index:
    tree_engine = create_query_engine_with_tree_summarize(index)

    if tree_engine:
        print("✅ Query engine with TreeSummarize created")

        # Test with a complex analytical query
        analytical_query = "Compare the advantages and disadvantages of different AI agent frameworks"
        print(f"\n🔍 Testing analytical query: '{analytical_query}'")

        # Now this will work:
        # response = tree_engine.query(analytical_query)
        # print(f"📝 TreeSummarize Response:\n{response}")
        print("   (Uncomment the query lines above to see the full synthesized answer)")
    else:
        print("❌ Failed to create TreeSummarize query engine")
else:
    print("❌ No index available - run previous cells first")


✅ Query engine with TreeSummarize created

🔍 Testing analytical query: 'Compare the advantages and disadvantages of different AI agent frameworks'
   (Uncomment the query lines above to see the full synthesized answer)


In [ ]:
from llama_index.core.output_parsers import PydanticOutputParser
from llama_index.core.program import LLMTextCompletionProgram
from pydantic import BaseModel, Field # Ensure BaseModel is imported
from typing import List

# Define the Pydantic model for structured output
class ResearchPaperInfo(BaseModel):
    title: str = Field(description="Main title or concept name")
    key_points: List[str] = Field(description="3-5 key points or findings")
    applications: List[str] = Field(description="Practical applications or use cases")
    summary: str = Field(description="2-3 sentence concise summary")

def create_structured_output_program(output_model: BaseModel = ResearchPaperInfo):
    """
    Create a structured output program using Pydantic models.

    Args:
        output_model: Pydantic model class for structured output

    Returns:
        LLMTextCompletionProgram that returns structured data
    """
    # Create output parser with the Pydantic model
    output_parser = PydanticOutputParser(output_cls=output_model)

    # Create the structured output program
    program = LLMTextCompletionProgram.from_defaults(
        output_parser=output_parser,
        prompt_template_str=(
            "You are an AI assistant that extracts structured information about "
            "AI research papers or AI concepts.\n\n"
            "Use the following CONTEXT to answer the QUERY and fill in these fields:\n"
            "- title: main title or concept name\n"
            "- key_points: 3-5 key points or findings\n"
            "- applications: practical applications or use cases\n"
            "- summary: 2-3 sentence concise summary\n\n"
            "CONTEXT:\n{context}\n\n"
            "QUERY:\n{query}\n\n"
            "Return ONLY a JSON object that matches the required structure."
        ),
    )

    return program

# Initialize the structured program
structured_program = create_structured_output_program()

# Define example context and query for testing
context = (
    "AI agents are designed to autonomously perform tasks in complex environments. "
    "They leverage advanced reasoning and planning capabilities, often using multiple tools "
    "and interacting with external data sources and APIs. "
    "Benefits include enhanced problem-solving, autonomous decision-making, and assistance in human tasks. "
    "Multi-agent architectures can further improve performance through parallel task execution and collaboration."
)
structure_query = "Extract key information about AI Agents."

response = structured_program(context=context, query=structure_query)
print("📊 Structured Response:", response)
print("Title:", response.title)
print("Key points:", response.key_points)

📊 Structured Response: title='AI Agents' key_points=['AI agents autonomously perform tasks in complex environments.', 'They utilize advanced reasoning and planning capabilities.', 'AI agents can interact with external data sources and APIs.', 'Multi-agent architectures enable parallel task execution and collaboration.', 'They enhance problem-solving and autonomous decision-making.'] applications=['Autonomous decision-making in dynamic environments', 'Enhanced problem-solving in complex tasks', 'Assistance in human tasks through automation', 'Collaboration in multi-agent systems for improved performance'] summary='AI agents are designed to autonomously perform tasks in complex environments by leveraging advanced reasoning and planning capabilities. They can interact with external data sources and APIs, and multi-agent architectures allow for parallel task execution and collaboration, enhancing problem-solving and decision-making.'
Title: AI Agents
Key points: ['AI agents autonomously pe

In [ ]:
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.response_synthesizers import TreeSummarize

def create_advanced_rag_pipeline(index, similarity_cutoff: float = 0.3, top_k: int = 10):
    """
    Create a comprehensive advanced RAG pipeline combining multiple techniques.

    Args:
        index: Vector index to query
        similarity_cutoff: Minimum similarity score for filtering
        top_k: Number of initial results to retrieve

    Returns:
        Advanced query engine with filtering and synthesis combined
    """
    # Create similarity postprocessor
    similarity_processor = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)

    # Create TreeSummarize for comprehensive responses
    tree_synthesizer = TreeSummarize()

    # Create the comprehensive query engine combining both techniques
    advanced_engine = index.as_query_engine(
        similarity_top_k=top_k,
        node_postprocessors=[similarity_processor],
        response_synthesizer=tree_synthesizer,
    )

    return advanced_engine

# Test the function
if index:
    advanced_pipeline = create_advanced_rag_pipeline(index, similarity_cutoff=0.3)

    if advanced_pipeline:
        print("✅ Advanced RAG pipeline created")

        # Define a complex query for testing
        complex_query = "What are the key differences between various types of AI agents described in the documents?"
        print(f"\n🔍 Testing complex query: '{complex_query}'")

        response = advanced_pipeline.query(complex_query)
        print(f"🚀 Advanced RAG Response:\n{response}")
    else:
        print("❌ Failed to create advanced RAG pipeline")
else:
    print("❌ No index available - run previous cells first")

✅ Advanced RAG pipeline created

🔍 Testing complex query: 'What are the key differences between various types of AI agents described in the documents?'
🚀 Advanced RAG Response:
The key differences between the various types of AI agents are based on their design and functionality:

1. **Autonomous Agents**: These agents are designed for independent task execution, handling complex tasks without external input. They are characterized by high complexity and a steep learning curve, making them suitable for tasks that require autonomy.

2. **Tool-Using Agents**: These agents are equipped to interact with external tools and data sources, which enhances their capabilities in tasks like document understanding and retrieval-augmented generation. They are generally easier to learn and have moderate to low complexity.

3. **Multi-Agent Systems**: These involve multiple agents working collaboratively, either in a vertical structure with a lead agent or a horizontal structure with equal collaborati

In [ ]:
# Final comparison: Basic vs Advanced RAG
print("🚀 Advanced RAG Techniques Assignment - Final Test")
print("=" * 60)

# Test queries for comparison
test_queries = [
    "What are the key capabilities of AI agents?",
    "How do you evaluate agent performance metrics?",
    "Explain the benefits and challenges of multimodal AI systems"
]

# Check if all components were created
components_status = {
    "Basic Index": index is not None,
    "Similarity Filter": 'filtered_engine' in locals() and filtered_engine is not None,
    "TreeSummarize": 'tree_engine' in locals() and tree_engine is not None,
    "Structured Output": 'structured_program' in locals() and structured_program is not None,
    "Advanced Pipeline": 'advanced_pipeline' in locals() and advanced_pipeline is not None
}

print("\n📊 Component Status:")
for component, status in components_status.items():
    status_icon = "✅" if status else "❌"
    print(f"   {status_icon} {component}")

# Create basic query engine for comparison
if index:
    print("\n🔍 Creating basic query engine for comparison...")
    basic_engine = index.as_query_engine(similarity_top_k=5)

    print("\n" + "=" * 60)
    print("🆚 COMPARISON: Basic vs Advanced RAG")
    print("=" * 60)

    for i, query in enumerate(test_queries, 1):
        print(f"\n📋 Test Query {i}: '{query}'")
        print("-" * 50)

        # Basic RAG
        print("🔹 Basic RAG:")
        if basic_engine:
            # Uncomment when testing:
            # basic_response = basic_engine.query(query)
            # print(f"   Response: {str(basic_response)[:200]}...")
            print("   (Standard vector search + simple response)")

        # Advanced RAG (if implemented)
        print("\n🔸 Advanced RAG:")
        if components_status["Advanced Pipeline"]:
            # Uncomment when testing:
            # advanced_response = advanced_pipeline.query(query)
            # print(f"   Response: {advanced_response}")
            print("   (Filtered + TreeSummarize + Structured output)")
        else:
            print("   Complete the advanced pipeline function to test")

# Final status
print("\n" + "=" * 60)
print("🎯 Assignment Status:")
completed_count = sum(components_status.values())
total_count = len(components_status)

print(f"   Completed: {completed_count}/{total_count} components")

if completed_count == total_count:
    print("\n🎉 Congratulations! You've mastered Advanced RAG Techniques!")
    print("   ✅ Node postprocessors for result filtering")
    print("   ✅ Response synthesizers for better answers")
    print("   ✅ Structured outputs for reliable data")
    print("   ✅ Advanced pipelines combining all techniques")
    print("\n🚀 You're ready for production RAG systems!")
else:
    missing = total_count - completed_count
    print(f"\n📝 Complete {missing} more components to finish the assignment:")
    for component, status in components_status.items():
        if not status:
            print(f"   - {component}")

print("\n💡 Key learnings:")
print("   - Postprocessors improve result relevance and precision")
print("   - Different synthesizers work better for different query types")
print("   - Structured outputs enable reliable system integration")
print("   - Advanced techniques can be combined for production systems")

🚀 Advanced RAG Techniques Assignment - Final Test

📊 Component Status:
   ✅ Basic Index
   ✅ Similarity Filter
   ✅ TreeSummarize
   ✅ Structured Output
   ✅ Advanced Pipeline

🔍 Creating basic query engine for comparison...

🆚 COMPARISON: Basic vs Advanced RAG

📋 Test Query 1: 'What are the key capabilities of AI agents?'
--------------------------------------------------
🔹 Basic RAG:
   (Standard vector search + simple response)

🔸 Advanced RAG:
   (Filtered + TreeSummarize + Structured output)

📋 Test Query 2: 'How do you evaluate agent performance metrics?'
--------------------------------------------------
🔹 Basic RAG:
   (Standard vector search + simple response)

🔸 Advanced RAG:
   (Filtered + TreeSummarize + Structured output)

📋 Test Query 3: 'Explain the benefits and challenges of multimodal AI systems'
--------------------------------------------------
🔹 Basic RAG:
   (Standard vector search + simple response)

🔸 Advanced RAG:
   (Filtered + TreeSummarize + Structured output